In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import math
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [0]:
def forward_stepwise_selection_BIC(X, y, BIC_diff):
  # start with all variables in the model
  # try dropping each variable one by one from the model
  # drop the variable that would lead to the greatest improvement
  # next, try dropping the remaining variables in the model one by one
  # drop the variable that would lead to the greatest improvement
  # repeat this process until it is no longer possible to improve the model
  
  # X: dataframe with explanatory variables
  # y: variable we are trying to predict
  X_variables = list(X.columns)
  good_vars = []
  good_vars_bic = []
  model = sm.OLS(endog=y, exog=X, missing='drop') #X.astype(float))
  results = model.fit()
  old_BIC =  np.inf
  counter = 20
  while len(X_variables) > 0 and counter > 0: # model is improving and there is at least one explanatory variable
    print("counter = {}".format(counter))
    counter -= 1
    current_best_bic = np.inf
    best_variable_to_add = None
    for variable in X_variables:
      model = sm.OLS(endog=y, exog=X[good_vars + [variable]], missing='drop')
      results = model.fit()
      bic = results.bic
      if bic < current_best_bic:
        current_best_bic = bic
        best_variable_to_add = variable
    if (old_BIC - current_best_bic) > BIC_diff: 
      print("best variable to add = {}".format(best_variable_to_add))
#       print(current_best_bic)
      X_variables.remove(best_variable_to_add)
      good_vars.append(best_variable_to_add)
      good_vars_bic.append(current_best_bic)
      old_BIC = current_best_bic
      print(old_BIC)
    else:
      return good_vars, good_vars_bic
  return good_vars, good_vars_bic

Beer Data

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [22]:
data = pd.read_csv('gdrive/My Drive/Project2_DATA_401/beer.csv')
data.head()

,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,review/timeStruct,review/timeUnix,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
0,40163,5.0,46634,14338,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,"{'min': 38, 'hour': 3, 'mday': 16, 'sec': 10, ...",1229398690,NaN,NaN,NaN,NaN,RblWthACoz
1,8135,11.0,3003,395,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,"{'min': 38, 'hour': 23, 'mday': 8, 'sec': 58, ...",1218238738,NaN,NaN,NaN,NaN,BeerSox
2,10529,4.7,961,365,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,"{'min': 7, 'hour': 18, 'mday': 26, 'sec': 2, '...",1101492422,NaN,NaN,NaN,Male,mschofield
3,44610,4.4,429,1,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,"{'min': 7, 'hour': 1, 'mday': 20, 'sec': 5, 'y...",1308532025,1.209827e+09,"Aug 10, 1976",208508400.0,Male,molegar76
4,37062,4.4,4904,1417,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,"{'min': 51, 'hour': 6, 'mday': 12, 'sec': 48, ...",1299912708,NaN,NaN,NaN,NaN,Brewbro000


In [47]:
targets = ["review/appearance", "review/aroma", "review/overall", "review/palate", "review/taste"]
predictors = ['beer/ABV', 'review/timeUnix', "beer/beerId", 'beer/brewerId', 'beer/style']
predictor_data = data[['beer/ABV', 'review/timeUnix', 'review/timeUnix']] # need to convert birthday to timestampe
data["review/text"] = ["" if isinstance(x, float) and math.isnan(x) else x.replace("\t", " ") for x in data["review/text"]] 
predictor_data.head()

,beer/ABV,review/timeUnix,review/timeUnix
0,5.0,1229398690,1229398690
1,11.0,1218238738,1218238738
2,4.7,1101492422,1101492422
3,4.4,1308532025,1308532025
4,4.4,1299912708,1299912708


In [48]:
beer_id_dummy = pd.get_dummies(data['beer/beerId'])
beer_id_dummy = beer_id_dummy.add_prefix('beer_id_')
beer_id_dummy.head()

,beer_id_175,beer_id_176,beer_id_178,beer_id_429,beer_id_436,beer_id_454,beer_id_503,beer_id_505,beer_id_507,beer_id_508,...,beer_id_76963,beer_id_76995,beer_id_76996,beer_id_76997,beer_id_76998,beer_id_76999,beer_id_77116,beer_id_77198,beer_id_77199,beer_id_77207
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
brewer_id_dummy = pd.get_dummies(data['beer/brewerId'])
brewer_id_dummy = brewer_id_dummy.add_prefix('brewer_id_')
brewer_id_dummy.head()

,brewer_id_1,brewer_id_14,brewer_id_60,brewer_id_163,brewer_id_263,brewer_id_289,brewer_id_365,brewer_id_394,brewer_id_395,brewer_id_453,...,brewer_id_26409,brewer_id_26612,brewer_id_26816,brewer_id_26946,brewer_id_26983,brewer_id_26990,brewer_id_27021,brewer_id_27079,brewer_id_27133,brewer_id_27797
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
beer_style_dummy = pd.get_dummies(data['beer/style'])
beer_style_dummy.head()

,Altbier,American Adjunct Lager,American Amber / Red Ale,American Amber / Red Lager,American Barleywine,American Black Ale,American Blonde Ale,American Brown Ale,American Dark Wheat Ale,American Double / Imperial IPA,...,Scotch Ale / Wee Heavy,Scottish Ale,Scottish Gruit / Ancient Herbed Ale,Smoked Beer,Tripel,Vienna Lager,Weizenbock,Wheatwine,Winter Warmer,Witbier
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
predictor_data = pd.merge(predictor_data, beer_style_dummy, left_index=True, right_index=True)
predictor_data = pd.merge(predictor_data, beer_id_dummy, left_index=True, right_index=True)
predictor_data = pd.merge(predictor_data, brewer_id_dummy, left_index=True,right_index=True)
predictor_data.head()

,beer/ABV,review/timeUnix,review/timeUnix,Altbier,American Adjunct Lager,American Amber / Red Ale,American Amber / Red Lager,American Barleywine,American Black Ale,American Blonde Ale,...,brewer_id_26409,brewer_id_26612,brewer_id_26816,brewer_id_26946,brewer_id_26983,brewer_id_26990,brewer_id_27021,brewer_id_27079,brewer_id_27133,brewer_id_27797
0,5.0,1229398690,1229398690,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11.0,1218238738,1218238738,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4.7,1101492422,1101492422,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.4,1308532025,1308532025,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.4,1299912708,1299912708,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
# list of review text
text = data["review/text"]
# create the transform
vectorizer = TfidfVectorizer(stop_words = ENGLISH_STOP_WORDS, max_features=50, max_df=0.85,smooth_idf=True,use_idf=True)
vectorizer.fit(text)
vector = vectorizer.transform(text)

In [0]:
words = list(vectorizer.vocabulary_.keys())
vectorized_data = pd.DataFrame(vector.toarray(), columns = words)

In [0]:
X = pd.merge(predictor_data, vectorized_data, left_index=True, right_index=True)#pd.concat([data, vectorized_data], axis=1, ignore_index=True)
X["log(ABV)"] = np.log(data["beer/ABV"])
X["(ABV)^2"] = data["beer/ABV"] ** 2
y = data["review/overall"]

In [56]:
# Need to re-run with no limit on how many features to select (will take a long time though)
good_vars, bic = forward_stepwise_selection_BIC(X, y, 100)

X_variables = ['beer/ABV', 'review/timeUnix', 'review/timeUnix', 'Altbier', 'American Adjunct Lager', 'American Amber / Red Ale', 'American Amber / Red Lager', 'American Barleywine', 'American Black Ale', 'American Blonde Ale', 'American Brown Ale', 'American Dark Wheat Ale', 'American Double / Imperial IPA', 'American Double / Imperial Pilsner', 'American Double / Imperial Stout', 'American IPA', 'American Malt Liquor', 'American Pale Ale (APA)', 'American Pale Lager', 'American Pale Wheat Ale', 'American Porter', 'American Stout', 'American Strong Ale', 'American Wild Ale', 'Baltic Porter', 'Belgian Dark Ale', 'Belgian IPA', 'Belgian Pale Ale', 'Belgian Strong Dark Ale', 'Belgian Strong Pale Ale', 'Berliner Weissbier', 'BiÃ¨re de Garde', 'Black & Tan', 'Bock', 'Braggot', 'California Common / Steam Beer', 'Chile Beer', 'Cream Ale', 'Czech Pilsener', 'Doppelbock', 'Dortmunder / Export Lager', 'Dubbel', 'Dunkelweizen', 'Eisbock', 'English Barleywine', 'English Bitter', 'English Brown Al

In [0]:
"""
Output of Feature Selection with only the 20 best Features
old_BIC = inf
counter = 20
best variable to add = review/timeUnix
81994.79083443646
counter = 19
best variable to add = log(ABV)
80019.29634033552
counter = 18
best variable to add = beer/ABV
78480.28112060089
counter = 17
best variable to add = American Double / Imperial Stout
77553.31354585008
counter = 16
best variable to add = roasted
76772.88785542494
counter = 15
best variable to add = coffee
76031.85409499507
counter = 14
best variable to add = smell
75353.65559595295
counter = 13
best variable to add = brewer_id_12224
74809.42572955527
counter = 12
best variable to add = brewer_id_765
74378.65946645789
counter = 11
best variable to add = American Malt Liquor
74030.15757782938
counter = 10
best variable to add = (ABV)^2
73698.69939626376
counter = 9
best variable to add = brewer_id_1199
73300.59777086921
counter = 8
best variable to add = beer_id_59239
72897.1886194611
counter = 7
best variable to add = Fruit / Vegetable Beer
72600.2840812654
counter = 6
best variable to add = Russian Imperial Stout
72302.03091523591
counter = 5
best variable to add = Euro Pale Lager
72041.48383930413
counter = 4
best variable to add = Munich Helles Lager
71815.8292974287
counter = 3
best variable to add = beer_id_429
71593.84076320872
counter = 2
best variable to add = brewer_id_14
71409.47166079063
counter = 1
best variable to add = beer_id_58053
71266.83749655793
"""